# Problema da mochila resolvido pelo algorítmo genético
---
* Equipe:
    * Gustavo dos Santos Melo
    * Luciano Douglas Machado Chagas
* Professor (a):
    * Adolfo Pinto Guimarães 

## 1. Instala as bibliotecas necessárias

Como o Google Colaboratory não mantém armazenada as bibliotecas que foram instaladas pelo usuário, e precisamos das bibliotecas **deap**, **matplotlib** e **numpy** instaladas. Elaboramos o código á baixo que faz o seguinte processo:

1. Executa o comando ```!pip freeze > requirements.txt```, que é responsável por listar todas as bibliotecas instaladas no ambiente no arquivo ```requirements.txt```;

2. Inicializa uma lista de bibliotecar requeridas, que são **deap**, **matplotlib** e **numpy**;

3. Abre o arquivo ```requirements.txt``` gerado no modo leitura;

4. Percorre cada linha do arquivo aberto;

    4.1. Como percebemos que o arquivo tem o padrão ```nome == versão``` na listagem das bibliotecas, na linha atual, quebramos em uma lista de string usando como separador os caractere ```==```;
    
    4.2. Á partir da separação do nome da biblioteca, verificamos se o nome dessa biblioteca está na lista de bibliotecas requeridas, caso esteja remove-se na lista de bibliotecas requeridas o nome da biblioteca atual;

5. Após a remoção do nome das bibliotecas que já se encontram instaladas, é encerrado o processo de leitura do arquivo e instalado as bibliotecas que ainda estão instalados.

In [ ]:
!pip freeze > requirements.txt

bibliotecas_requeridas = [
    "deap",
    "matplotlib",
    "numpy"
]

with open(file="./requirements.txt", mode="r") as arquivo:
    for linha in arquivo:
        pedaco_linha = linha.split("==")

        if pedaco_linha[0] in bibliotecas_requeridas:
            bibliotecas_requeridas.remove(pedaco_linha[0])

    arquivo.close()

if len(bibliotecas_requeridas) == 0:
    print("Requisição de bibliotecas já satisfeita!")
else:
    if "deap" in bibliotecas_requeridas:
        !pip install deap

    if "matplotlib" in bibliotecas_requeridas:
        !pip install matplotlib

    if "numpy" in bibliotecas_requeridas:
        !pip install numpy

## 2. Importa as bibliotecas necessárias

In [ ]:
import json
import numpy
import random

from deap import algorithms, base, creator, tools

## 3. Gera Itens aleatórios

In [ ]:
TAMANHO_POPULACAO = 5
MAXIMO_ITENS = 50
MAXIMO_PESO = 50
NUMERO_ITENS = 20

# Para poder reproduzir o mesmo experimento depois,
# precisa colocar um valor na semente (Seed)

random.seed(
    a=64
)

itens = {}

# Gera itens com peso e valor aleatório

for i in range(NUMERO_ITENS):
    itens[i] = (
        random.randint(
            a=1,
            b=10
        ),
        random.uniform(
            a=0,
            b=100
        )
    )

# Imprime os itens gerados

print(
    json.dumps(
        itens,
        indent= 2,
        sort_keys=True,
        ensure_ascii=True
    )
)

## 4. Gera a função de apitidão do tipo MinMax e os individuos

In [ ]:
# Gera função MinMax

creator.create(
    "FitnessMinMax",
    base.Fitness,
    weights=(
        -1.0,
        1.0
    )
)

# Gera Individuo

creator.create(
    "Mochila",
    set,
    fitness=creator.FitnessMinMax
)

## 5. Inicializa a mochila dos indivíduos, a população e registra as funções que vai ser ultilizado pelo eaMuPlusLambda

In [ ]:
def calc_mochila(mochila):
    peso = 0.0
    valor = 0.0
    for item in mochila:
        peso += itens[item][0]
        valor += itens[item][1]
    
    # Se a mochila não for válida, retorne o pior "possível":
    # algo pesado e sem valor
    if len(mochila) > MAXIMO_ITENS or peso > MAXIMO_PESO:
        return 10000, 0 

    return peso, valor

In [ ]:
def crossover(individuo_1, individuo_2):
    """
    Como estamos usando Conjuntos, a função de crossover é feita com
    os operadores &= e ^= de conjuntos.
    & = : Intersecção entre Conjunto 1 e 2
    ^ = : Diferença Simétrica entre Conjunto 1 e 2
    """
    temp = set(individuo_1)
    individuo_1 &= individuo_2
    individuo_2 ^= temp
    return individuo_1, individuo_2

In [ ]:
def mutacao(individuo):
    """Mutacao adiciona ou remove uma Mochila com 50% de chance"""
    if random.random() < 0.5:
        if len(individuo) > 0:     # We cannot pop from an empty set
            individuo.remove(random.choice(sorted(tuple(individuo))))
    else:
        individuo.add(random.randrange(NUMERO_ITENS))

    return individuo,

In [ ]:
# Inicializa o Toolbox

toolbox = base.Toolbox()

# Registra um numero aleatório de no máximo NUMERO_ITENS

toolbox.register(
    "atributo",
    random.randrange,
    NUMERO_ITENS
)

# Inicializa a mochila

toolbox.register(
    "mochila", 
    tools.initRepeat, 
    creator.Mochila, 
    toolbox.atributo,
    TAMANHO_POPULACAO
)

# Inicializa a população

toolbox.register(
    "population",
    tools.initRepeat, 
    list, 
    toolbox.mochila
)

# Registrando as funcoes que necessárias

toolbox.register(
    "evaluate",
    calc_mochila
)

toolbox.register(
    "mate",
    crossover
)

toolbox.register(
    "mutate",
    mutacao
)

toolbox.register(
    "select",
    tools.selNSGA2
)

# 6. Gera e imprime dados estatísticos

In [ ]:
GERACOES = 50   # quantas geracoes ou epocas serao geradas
MU = 50         # tamanho da populacao
LAMBDA = 100    # tamanho da recombinacao
CXPB = 0.7      # porcentagem da populacao que vai sofrer crossover
MUTPB = 0.2     # porcentagem da populacao que vai sofrer mutacao

pop = toolbox.population(n=MU)
hof = tools.ParetoFront()

stats = tools.Statistics(lambda ind: ind.fitness.values)

stats.register("avg", numpy.mean, axis=0)
stats.register("std", numpy.std, axis=0)
stats.register("min", numpy.min, axis=0)
stats.register("max", numpy.max, axis=0)

final = algorithms.eaMuPlusLambda(
    pop, 
    toolbox, 
    MU, 
    LAMBDA,
    CXPB, 
    MUTPB, 
    GERACOES, 
    stats, 
    halloffame=hof
)

In [ ]:
print("Populacao final:")
for item in pop:
  print(item)

In [ ]:
print("Estatistica Final:")
print(stats.compile(pop))

In [ ]:
for item in reversed(hof):
  print(f"{item} Peso: {calc_mochila(item)[0]}, Valor: {calc_mochila(item)[1]}")